In [122]:
import math

In [125]:
a1 = -1
a2 = 2
b1 = -2
b2 = 2
cols = 20
rows = 20

cur_error = 1
error = 0.0001

x_step = (a2 - a1)/cols
y_step = (b2 - b1)/rows

def phi(x, y):
    return math.exp(1 - (x + y)*(x + y))

def F(x, y):
    tmp = (x + y)*(x + y)
    return 4 * (-2*tmp + 1) * math.exp(1-tmp)

def X(i):
    return a1 + i * x_step

def Y(i):
    return b1 + i * y_step

def cell(m, row, col):
    return m[row * cols + col]

def set_cell(m, row, col, value):
    m[row * cols + col] = value
    
def scalar(a, b):
    ret = 0
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            ret += cell(a, i, j) * cell(b, i, j) * (x_step * y_step)**2
    return ret

def print_matrix(m):
    for i in range(rows - 1, -1, -1):
        buf = []
        for j in range(cols):
            buf.append(cell(m, i, j))
        print(buf)

# лаплас со знаком -
def laplas_5_matrix(m):
    ret = m.copy()
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            a11 = cell(m, i, j)
            a21 = cell(m, i + 1, j)
            a01 = cell(m, i - 1, j)
            a10 = cell(m, i, j - 1)
            a12 = cell(m, i, j + 1)
            tmp1 = (2*a11 - a01 - a21)/(y_step * y_step)
            tmp2 = (2*a11 - a10 - a12)/(x_step * x_step)
            set_cell(ret, i, j, (tmp1 + tmp2))
    return ret

P = [0] * (cols * rows)

# R заполнили нулями
R = P.copy()

# Заполнили P граничными значениями, внутренние значения 0
for i in range(0, cols):
    set_cell(P, 0, i, phi(X(i), Y(0)))
    set_cell(P, rows - 1, i, phi(X(i), Y(rows - 1)))
    
for i in range(0, rows):
    set_cell(P, i, 0, phi(X(0), Y(i)))
    set_cell(P, i, cols - 1, phi(X(cols - 1), Y(i)))


def calculate_next_R():
    matrix_buffer_cur = laplas_5_matrix(P)
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            set_cell(R, i, j, cell(matrix_buffer_cur, i, j) - F(X(j), Y(i)))
    for i in range(0, cols):
        set_cell(R, 0, i, 0)
        set_cell(R, rows - 1, i, 0)
    for i in range(0, rows):
        set_cell(R, i, 0, 0)
        set_cell(R, i, cols - 1, 0)

def calculate_next_P(tau, r):
    for i in range(rows):
        for j in range(cols):
            old = cell(P, i, j)
            set_cell(P, i, j, old - tau * cell(r, i, j))
            
# r_0; по границам нули
calculate_next_R() 

count = 1
matrix_buffer1 = [0] * (cols * rows)
matrix_buffer2 = [0] * (cols * rows)
old = [0] * (cols * rows)
while (cur_error >= error):
    #print (count)
    for i in range(rows):
        for j in range(cols):
            set_cell(old, i,j, cell(P, i, j))
    matrix_buffer = laplas_5_matrix(R)
    tmp1 = scalar(matrix_buffer, R)  #r_k
    tmp2 = scalar(matrix_buffer, matrix_buffer)
    tau = tmp1 / tmp2
    calculate_next_P(tau, R)   #p_k+1
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            set_cell(matrix_buffer1, i, j, cell(P, i, j) - F(X(j), Y(i)))
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            set_cell(matrix_buffer2, i, j, cell(P, i, j) - cell(old, i, j))
    cur_error = math.sqrt(scalar(matrix_buffer2,matrix_buffer2))
    global_error = math.sqrt(scalar(matrix_buffer1, matrix_buffer1))
    calculate_next_R()
    count += 1
   # print(cur_error)
   # print(global_error)

2.0
